In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('../input/titanic/train.csv')
test_data = pd.read_csv('../input/titanic/test.csv')

In [3]:
train_data.shape

(891, 12)

In [4]:
#returns a dataframe with only entries that have a null values in a particular field
def null_entries(field):
    null_values = train_data[[field]].isnull()
    null_values = null_values[null_values[field] == True]
    return null_values

In [23]:
print(null_entries('Embarked'))

     Embarked
61       True
829      True


In [5]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
import seaborn as sns
sns.barplot(x=train_data['Survived'],y = train_data['Age'],hue = train_data['Pclass'])

In [6]:
#model imports
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer

In [7]:
#data splitting and processing
y = train_data['Survived']
x = train_data.copy()
x = x.drop(['Survived'], axis = 1)
print(x.dtypes)
print(y.dtypes)
x_train, x_valid, y_train, y_valid = train_test_split(x,y,test_size = 0.3)

print(f'x train shape {x_train.shape}')
print(f'x valid shape {x_valid.shape}')
print(f'y train shape {y_train.shape}')
print(f'y valid shape {y_valid.shape}')

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
int64
x train shape (623, 11)
x valid shape (268, 11)
y train shape (623,)
y valid shape (268,)


In [ ]:
#defining pipelines
age_imputer = SimpleImputer(strategy = 'mean')
cabin_imputer = SimpleImputer(strategy = 'constant', fill_value = 'NaN')
embarked_imput = SimpleImputer(strategy = 'constant', fill_value = 'None')
trans = ColumnTransformer(transformers = [('age', age_imputer, ['Age']),('cabin', cabin_imputer, ['Cabin']), ('embarked', embarked_imputer, ['Embarked'])])

In [ ]:
model = RandomForestClassifier(n_estimators = 10)
pipe = Pipeline(steps = [('pre', trans), ('model', model)])
pipe.fit(x_train, y_train)